## Домашнее задание 5. Логистическая регрессия и случайный лес в задаче кредитного скоринга

В этом задании вы построите модели и ответите на вопросы, используя данные по кредитному скорингу.

## Постановка задачи кредитного скоринга

#### Задача

Предсказать, вернёт ли клиент кредит в течение 90 дней. Это задача бинарной классификации — мы будем относить клиентов к категориям «хороший» или «плохой» на основе нашего прогноза.

#### Описание данных

| Признак | Тип переменной | Тип значений | Описание |
|:--------|:--------------|:-----------|:----------|
| age | Входной признак | integer | Возраст клиента |
| DebtRatio | Входной признак | real | Общие ежемесячные платежи по кредитам (кредит, алименты и т.д.) / Общий ежемесячный доход в процентах |
| NumberOfTime30-59DaysPastDueNotWorse | Входной признак | integer | Количество случаев просрочки 30–59 дней (не хуже) по другим кредитам за последние 2 года |
| NumberOfTimes90DaysLate | Входной признак | integer | Количество случаев просрочки 90+ дней по другим кредитам |
| NumberOfTime60-89DaysPastDueNotWorse | Входной признак | integer | Количество случаев просрочки 60–89 дней (не хуже) за последние 2 года |
| NumberOfDependents | Входной признак | integer | Количество иждивенцев клиента |
| SeriousDlqin2yrs | Целевая переменная | binary: <br>0 или 1 | Клиент не погасил долг по кредиту в течение 90 дней |

Настроим окружение:

In [ ]:
# Отключение предупреждений
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 11, 8

Напишем функцию, которая заменит значения *NaN* медианой по каждому столбцу.

In [ ]:
def fill_nan(table):
    for col in table.columns:
        table[col] = table[col].fillna(table[col].median())
    return table   

Прочитаем данные:

In [ ]:
data = pd.read_csv('../data/credit_scoring_sample.csv')
data.head()

Посмотрим на типы переменных:

In [ ]:
data.dtypes

Проверим баланс классов:

In [ ]:
ax = data['SeriousDlqin2yrs'].hist(orientation='horizontal', color='red')
ax.set_xlabel("number_of_observations")
ax.set_ylabel("unique_value")
ax.set_title("Target distribution")

print('Distribution of the target:')
data['SeriousDlqin2yrs'].value_counts()/data.shape[0]

Выделим имена входных переменных, исключив целевую:

In [ ]:
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']
independent_columns_names

Применим функцию для замены значений *NaN*:

In [ ]:
table = fill_nan(data)

Разделим целевую переменную и входные признаки:

In [ ]:
X = table[independent_columns_names]
y = table['SeriousDlqin2yrs']

## Бутстрэп

**Вопрос 1.** Постройте интервальную оценку среднего возраста клиентов, просрочивших выплату, с уровнем доверия 90%. Используйте `np.random.seed(0)`. Какой получился доверительный интервал?

1. 52.59 – 52.86
2. 45.71 – 46.13
3. 45.68 – 46.17
4. 52.56 – 52.88

In [ ]:
# ваш код здесь

## Логистическая регрессия

Настроим логистическую регрессию:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

Создадим модель `LogisticRegression` с `class_weight='balanced'` для компенсации несбалансированности классов.

In [ ]:
lr = LogisticRegression(random_state=5, class_weight='balanced')

Попробуем найти лучший коэффициент регуляризации — параметр `C` логистической регрессии. Это позволит получить оптимальную модель без переобучения, хорошо предсказывающую целевую переменную.

In [ ]:
parameters = {'C': (0.0001, 0.001, 0.01, 0.1, 1, 10)}

Для нахождения оптимального значения `C` используем стратифицированную 5-fold кросс-валидацию и посмотрим на *ROC AUC* при различных значениях `C`. Используем функцию `StratifiedKFold`:

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

Одна из важных метрик качества модели — *Area Under the Curve (AUC)*. *ROC AUC* принимает значения от 0 до 1. Чем ближе ROC AUC к 1, тем лучше качество классификационной модели.

**Вопрос 2.** Выполните *Grid Search* с метрикой "roc_auc" по параметру `C`. Какое значение `C` оптимально?

1. 0.0001
2. 0.001
3. 0.01
4. 0.1
5. 1
6. 10

In [ ]:
# ваш код здесь

**Вопрос 3.** Можно ли считать лучшую модель стабильной? Модель *стабильна*, если стандартное отклонение на валидации менее 0.5%. Сохраните значение *ROC AUC* лучшей модели — оно понадобится далее.

1. Да
2. Нет

In [ ]:
# ваш код здесь

## Важность признаков

**Вопрос 4.** *Важность признака* определяется абсолютным значением его соответствующего коэффициента. Сначала нужно нормализовать все значения признаков, чтобы их можно было корректно сравнивать. Какой признак наиболее важен для лучшей модели логистической регрессии?

1. age
2. NumberOfTime30-59DaysPastDueNotWorse
3. DebtRatio
4. NumberOfTimes90DaysLate
5. NumberOfTime60-89DaysPastDueNotWorse
6. MonthlyIncome
7. NumberOfDependents

In [ ]:
# ваш код здесь

**Вопрос 5.** Рассчитайте, насколько *DebtRatio* влияет на предсказание, используя [функцию softmax](https://en.wikipedia.org/wiki/Softmax_function). Какое значение получается?

1. 0.38
2. -0.02
3. 0.11
4. 0.24

In [ ]:
# ваш код здесь

**Вопрос 6.** Рассмотрим интерпретацию влияния признаков. Для этого пересчитайте логистическую регрессию с абсолютными значениями, то есть без масштабирования. Затем увеличьте возраст клиента на 20 лет, оставив остальные признаки без изменений. Во сколько раз увеличится вероятность того, что клиент не вернёт долг? Пример теоретического расчёта можно найти [здесь](https://www.unm.edu/~schrader/biostat/bio2/Spr06/lec11.pdf).

1. -0.01
2. 0.70
3. 8.32
4. 0.66

In [ ]:
# ваш код здесь

## Случайный лес

Импортируем классификатор случайного леса:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

Инициализируем случайный лес со 100 деревьями и балансировкой целевых классов:

In [ ]:
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42, 
                            class_weight='balanced')

Будем искать лучшие параметры среди следующих значений:

In [ ]:
parameters = {'max_features': [1, 2, 4], 'min_samples_leaf': [3, 5, 7, 9], 'max_depth': [5,10,15]}

Снова используем стратифицированную кросс-валидацию. Переменная `skf` должна быть ещё доступна.

**Вопрос 7.** Насколько *ROC AUC* лучшей модели случайного леса выше, чем у лучшей логистической регрессии на валидации?

1. 4%
2. 3%
3. 2%
4. 1%

In [ ]:
# ваш код здесь

**Вопрос 8.** Какой признак имеет наименьшее влияние в модели случайного леса?

1. age
2. NumberOfTime30-59DaysPastDueNotWorse
3. DebtRatio
4. NumberOfTimes90DaysLate
5. NumberOfTime60-89DaysPastDueNotWorse
6. MonthlyIncome
7. NumberOfDependents

In [ ]:
# ваш код здесь

**Вопрос 9.** Какое главное преимущество *логистической регрессии* перед *случайным лесом* в данной задаче?

1. Меньше времени на обучение модели
2. Меньше переменных для перебора
3. Интерпретируемость признаков
4. Линейные свойства алгоритма

## Бэггинг

Импортируем модули и настроим параметры для бэггинга:

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

parameters = {'max_features': [2, 3, 4], 'max_samples': [0.5, 0.7, 0.9], 
              'estimator__C': [0.0001, 0.001, 0.01, 1, 10, 100]}

**Вопрос 10.** Обучите бэггинг-классификатор с `random_state=42`. В качестве базовых классификаторов используйте 100 логистических регрессий и `RandomizedSearchCV` вместо `GridSearchCV`. Перебор всех 54 вариантов займёт много времени, поэтому установите максимальное число итераций `RandomizedSearchCV` равным 20. Не забудьте указать параметры `cv` и `random_state=1`. Какое лучшее значение *ROC AUC* вы получили?

1. 80.75%
2. 80.12%
3. 79.62%
4. 76.50%

In [ ]:
# ваш код здесь

**Вопрос 11.** Дайте интерпретацию лучших параметров бэггинга. Почему именно такие значения `max_features` и `max_samples` оптимальны?

1. Для бэггинга важно использовать как можно меньше признаков
2. Бэггинг работает лучше на малых выборках
3. Меньше корреляция между отдельными моделями
4. Чем больше признаков, тем меньше потеря информации